In [57]:
import numpy as np
import pandas as pd

from dask import delayed
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [58]:
pdf = pd.read_csv('../data/charges.csv')
pdf = pd.concat([pdf] * 100)
pdf = pdf.reset_index()

ddf = dd.from_pandas(pdf, npartitions=2)

In [59]:
pdf

,index,charges,bmi,gender,smoker,region,children,last_visit
0,0,33467,37,male,yes,SE,1,2017-07-23
1,1,5023,20,female,yes,NW,3,2020-03-30
2,2,6713,24,male,no,NE,4,2018-01-31
3,3,10056,21,male,no,SE,4,2021-06-02
4,4,18994,35,male,yes,SE,5,2017-03-05
...,...,...,...,...,...,...,...,...
999995,9995,14397,26,male,no,NE,1,2018-09-28
999996,9996,22918,32,male,yes,SW,5,2021-11-24
999997,9997,4606,25,female,yes,NE,2,2017-12-16
999998,9998,3776,27,male,yes,NE,1,2017-01-08


In [60]:
@delayed
def slow_function(row):
    if row['smoker'] == 'yes':
        return np.power(row['charges'], 3)
    else:
        return np.power(row['charges'], 2)

In [61]:
# pandas dataframe (1M rows)
%timeit pdf.apply(lambda row: slow_function, axis=1)

6.57 s ± 157 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
# dask dataframe (1M rows, 10 partitions)
%timeit ddf.apply(lambda row: slow_function, axis=1, meta=(None, 'object')).compute()

12 s ± 552 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
